In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime

from dateutil.parser import parse

In [2]:
path = os.path.join('..', '..', 'data', 'interim', '2_tbs_with_datetime.csv')
df = pd.read_csv(path, index_col = 0)

/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head(5)

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter
0,712382,2017-01-01 00:00:00,2017-01-01 00:03:00,223.0,7051,Wellesley St E / Yonge St Green P,7089,Church St / Wood St,Member,6,0,1,1
1,712383,2017-01-01 00:00:00,2017-01-01 00:05:00,279.0,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member,6,0,1,1
2,712384,2017-01-01 00:05:00,2017-01-01 00:29:00,1394.0,7113,Parliament St / Aberdeen Ave,7199,College St W / Markham St,Member,6,0,1,1
3,712385,2017-01-01 00:07:00,2017-01-01 00:21:00,826.0,7077,College Park South,7010,King St W / Spadina Ave,Member,6,0,1,1
4,712386,2017-01-01 00:08:00,2017-01-01 00:12:00,279.0,7079,McGill St / Church St,7047,University Ave / Gerrard St W,Member,6,0,1,1


In [4]:
df['trip_start_time'] = pd.to_datetime(df['trip_start_time'], format='%Y-%m-%d %H:%M:%S')
df['trip_stop_time'] = pd.to_datetime(df['trip_stop_time'], format='%Y-%m-%d %H:%M:%S')

In [5]:
df.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds           float64
from_station_id                  object
from_station_name                object
to_station_id                    object
to_station_name                  object
user_type                        object
day                               int64
week                              int64
month                             int64
quarter                           int64
dtype: object

In [6]:
pd.set_option('display.max_rows', 1000)
df_rand = df.sample( n = 500, random_state = 1 ).copy()
df_rand = df_rand.sort_index(ascending=True)
#df_rand.head(500)

In [7]:
df_rand['check_time'] = (df_rand['trip_stop_time'] - df_rand['trip_start_time']) / np.timedelta64(1,'s')
df_rand.head(20)

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time
8642,722809,2017-01-10 14:30:00,2017-01-10 14:49:00,1113.0,7016,Bay St / Queens Quay W (Ferry Terminal),7072,Queen St W / Shaw St,Member,1,2,1,1,1140.0
8691,722889,2017-01-10 16:26:00,2017-01-10 16:33:00,447.0,7190,St. George St / Hoskin Ave,7027,Beverly St / Dundas St W,Member,1,2,1,1,420.0
10584,725133,2017-01-12 00:05:00,2017-01-12 00:10:00,286.0,7108,Front St E / Cherry St,7060,Princess St / Adelaide St,Member,3,2,1,1,300.0
14692,730005,2017-01-14 19:30:00,2017-01-14 19:35:00,310.0,7059,Front St W / Blue Jays Way,7048,Front St / Yonge St (Hockey Hall of Fame),Member,5,2,1,1,300.0
15556,731051,2017-01-15 21:18:00,2017-01-15 21:21:00,169.0,7078,College St W / Major St,7199,College St W / Markham St,Member,6,2,1,1,180.0
16978,732648,2017-01-16 21:47:00,2017-01-16 21:58:00,679.0,7153,Bloor St W / Christie St,7046,Niagara St / Richmond St W,Member,0,3,1,1,660.0
19210,735366,2017-01-18 15:16:00,2017-01-18 15:21:00,319.0,7161,Beverly St / College St W,7162,Hayter St / Laplante Ave,Member,2,3,1,1,300.0
25327,742622,2017-01-21 18:38:00,2017-01-21 18:52:00,792.0,7064,51 Parliament St,7051,Wellesley St E / Yonge St Green P,Member,5,3,1,1,840.0
26372,743762,2017-01-22 15:47:00,2017-01-22 15:56:00,556.0,7097,Riverdale Park North (Broadview Ave),7104,King St E / River St,Member,6,3,1,1,540.0
28723,746481,2017-01-23 23:35:00,2017-01-23 23:39:00,246.0,7057,Simcoe St / Wellington St W,7054,Navy Wharf Ct. / Bremner Blvd.,Member,0,4,1,1,240.0


In [8]:
df_rand['time_delta'] = (df_rand['check_time'] - df_rand['trip_duration_seconds'])
df_rand.tail()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta
1467632,2351929,2017-12-11 15:41:00,2017-12-11 16:02:00,1209.0,Riverdale Park North (Broadview Ave),Dundas St W / St. Patrick St,Member,NaN,NaN,0,50,12,4,1260.0,51.0
1476788,2363303,2017-12-17 12:25:53,2017-12-17 12:32:44,411.0,Front St W / Bay St (South Side),Navy Wharf Ct. / Bremner Blvd.,Member,NaN,NaN,6,50,12,4,411.0,0.0
1489105,2378871,2017-12-24 09:35:10,2017-12-24 09:51:46,996.0,The Royal Ontario Museum (Bloor St Entrance),Willcocks St / St. George St,Casual,NaN,NaN,6,51,12,4,996.0,0.0
1489384,2379292,2017-12-24 15:55:15,2017-12-24 16:00:58,343.0,Hanna Ave / Liberty St,Queen St W / Ossington Ave,Member,NaN,NaN,6,51,12,4,343.0,0.0
1491287,2382191,2017-12-29 12:09:32,2017-12-29 12:21:28,716.0,Salem Ave / Bloor St W,Yarmouth Rd / Christie St,Member,NaN,NaN,4,52,12,4,716.0,0.0


In [9]:
df_rand['time_flag'] = df_rand['time_delta'] > 500
df_rand['time_flag'].value_counts()

False    500
Name: time_flag, dtype: int64

# Conclude the trip duration is well inputed

## check to see if nan columns are well arranged

In [10]:
overage_arr = [1800, 3600, 5400, 7200, 9000]

In [11]:
df['nan_flag'] = (( df['to_station_name'].isnull()) & (df['user_type'].isnull()) ) | (( df['to_station_name'].notnull()) & (df['user_type'].notnull()))

In [12]:
df['nan_flag'].value_counts()

True    1492368
Name: nan_flag, dtype: int64

## only two ordering types

In [13]:
df_rand['nan_row_flag'] = ( df_rand['to_station_name'].isnull()) & (df_rand['user_type'].isnull()) 
df['nan_row_flag'] = ( df['to_station_name'].isnull()) & (df['user_type'].isnull()) 

In [14]:
df_rand.tail()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
1467632,2351929,2017-12-11 15:41:00,2017-12-11 16:02:00,1209.0,Riverdale Park North (Broadview Ave),Dundas St W / St. Patrick St,Member,NaN,NaN,0,50,12,4,1260.0,51.0,False,True
1476788,2363303,2017-12-17 12:25:53,2017-12-17 12:32:44,411.0,Front St W / Bay St (South Side),Navy Wharf Ct. / Bremner Blvd.,Member,NaN,NaN,6,50,12,4,411.0,0.0,False,True
1489105,2378871,2017-12-24 09:35:10,2017-12-24 09:51:46,996.0,The Royal Ontario Museum (Bloor St Entrance),Willcocks St / St. George St,Casual,NaN,NaN,6,51,12,4,996.0,0.0,False,True
1489384,2379292,2017-12-24 15:55:15,2017-12-24 16:00:58,343.0,Hanna Ave / Liberty St,Queen St W / Ossington Ave,Member,NaN,NaN,6,51,12,4,343.0,0.0,False,True
1491287,2382191,2017-12-29 12:09:32,2017-12-29 12:21:28,716.0,Salem Ave / Bloor St W,Yarmouth Rd / Christie St,Member,NaN,NaN,4,52,12,4,716.0,0.0,False,True


In [15]:
df_badcols = df_rand[df_rand['nan_row_flag'] == True]
df_badcols.loc[:,'user_type'] = df_badcols.loc[:,'to_station_id']
df_badcols.loc[:,'to_station_name'] = df_badcols.loc[:,'from_station_name']
df_badcols.loc[:,'to_station_id'] = np.nan
df_badcols.loc[:,'from_station_name'] = df_badcols.loc[:,'from_station_id']
df_badcols.loc[:,'from_station_id'] = np.nan

/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/pandas/core/indexing.py:964: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [16]:
df_badcols.tail()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
1467632,2351929,2017-12-11 15:41:00,2017-12-11 16:02:00,1209.0,NaN,Riverdale Park North (Broadview Ave),NaN,Dundas St W / St. Patrick St,Member,0,50,12,4,1260.0,51.0,False,True
1476788,2363303,2017-12-17 12:25:53,2017-12-17 12:32:44,411.0,NaN,Front St W / Bay St (South Side),NaN,Navy Wharf Ct. / Bremner Blvd.,Member,6,50,12,4,411.0,0.0,False,True
1489105,2378871,2017-12-24 09:35:10,2017-12-24 09:51:46,996.0,NaN,The Royal Ontario Museum (Bloor St Entrance),NaN,Willcocks St / St. George St,Casual,6,51,12,4,996.0,0.0,False,True
1489384,2379292,2017-12-24 15:55:15,2017-12-24 16:00:58,343.0,NaN,Hanna Ave / Liberty St,NaN,Queen St W / Ossington Ave,Member,6,51,12,4,343.0,0.0,False,True
1491287,2382191,2017-12-29 12:09:32,2017-12-29 12:21:28,716.0,NaN,Salem Ave / Bloor St W,NaN,Yarmouth Rd / Christie St,Member,4,52,12,4,716.0,0.0,False,True


In [17]:
df_rand.tail()
df_rand.index

Int64Index([   8642,    8691,   10584,   14692,   15556,   16978,   19210,
              25327,   26372,   28723,
            ...
            1460945, 1461077, 1462990, 1463276, 1463356, 1467632, 1476788,
            1489105, 1489384, 1491287],
           dtype='int64', length=500)

In [18]:
df_rand.loc[df_rand.index.isin(df_badcols.index), :] = df_badcols

In [19]:
df_rand.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
8642,722809,2017-01-10 14:30:00,2017-01-10 14:49:00,1113.0,7016,Bay St / Queens Quay W (Ferry Terminal),7072,Queen St W / Shaw St,Member,1,2,1,1,1140.0,27.0,False,False
8691,722889,2017-01-10 16:26:00,2017-01-10 16:33:00,447.0,7190,St. George St / Hoskin Ave,7027,Beverly St / Dundas St W,Member,1,2,1,1,420.0,-27.0,False,False
10584,725133,2017-01-12 00:05:00,2017-01-12 00:10:00,286.0,7108,Front St E / Cherry St,7060,Princess St / Adelaide St,Member,3,2,1,1,300.0,14.0,False,False
14692,730005,2017-01-14 19:30:00,2017-01-14 19:35:00,310.0,7059,Front St W / Blue Jays Way,7048,Front St / Yonge St (Hockey Hall of Fame),Member,5,2,1,1,300.0,-10.0,False,False
15556,731051,2017-01-15 21:18:00,2017-01-15 21:21:00,169.0,7078,College St W / Major St,7199,College St W / Markham St,Member,6,2,1,1,180.0,11.0,False,False


# Check i proper columns are 1 - 1

In [94]:
def isOneToOne(df, col1, col2):
    first = df.groupby(col1)[col2].nunique().max()
    second = df.groupby(col2)[col1].nunique().max()
    return first + second == 2

In [193]:
def get_relation(df, col1, col2):        
    first_max = df[[col1, col2]].groupby(col1)[col2].nunique().max()
    second_max = df[[col1, col2]].groupby(col2)[col1].nunique().max()
    print('maxes:', first_max, second_max)
    if first_max==1:
        if second_max==1:
            return 'one-to-one'
        else:
            return 'one-to-many'
    else:
        if second_max==1:
            return 'many-to-one'
        else:
            return 'many-to-many'

In [194]:
def get_relation_check(df, col1, col2):        
    first_max = df[[col1, col2]].groupby(col1)[col2].nunique()
    second_max = df[[col1, col2]].groupby(col2)[col1].nunique()
    return first_max, second_max

In [195]:
df_goodcols[df_goodcols['to_station_id'] == 7020]

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
41911,761920,2017-02-01 15:05:00,2017-02-01 15:10:00,309.0,7059,Front St W / Blue Jays Way,7020,Phoebe St / Spadina Ave,Member,2,5,2,1,300.0,-9.0,False,False
50577,772131,2017-02-07 16:10:00,2017-02-07 16:22:00,733.0,7208,80 Clinton St (North of College),7020,Phoebe St / Spadina Ave,Member,1,6,2,1,720.0,-13.0,False,False
136752,873823,2017-04-03 14:02:00,2017-04-03 14:13:00,611.0,7038,Dundas St / Yonge St,7020,Phoebe St / Spadina Ave,Member,0,14,4,2,660.0,49.0,False,False
197648,943472,2017-04-26 16:00:00,2017-04-26 16:03:00,180.0,7032,Augusta Ave / Dundas St W,7020,Phoebe St / Spadina Ave,Member,2,17,4,2,180.0,0.0,False,False
273555,1031750,2017-05-20 20:25:00,2017-05-20 20:50:00,1481.0,7007,College St W / Huron St,7020,Phoebe St / Spadina Ave,Casual,5,20,5,2,1500.0,19.0,False,False
276183,1034881,2017-05-22 13:27:00,2017-05-22 13:41:00,842.0,7153,Bloor St W / Christie St,7020,Phoebe St / Spadina Ave,Casual,0,21,5,2,840.0,-2.0,False,False
343985,1114155,2017-06-07 16:04:00,2017-06-07 16:14:00,549.0,7021,Bay St / Albert St,7020,Phoebe St / Spadina Ave,Member,2,23,6,2,600.0,51.0,False,False


In [196]:
df1, df2 = get_relation_check(df_goodcols, 'to_station_id', 'to_station_name')

In [197]:
df1.max() + df2.max()

2

In [198]:
s = df2.idxmax()
s

'161 Bleecker St (South of Wellesley)'

In [199]:
df_goodcols[df_goodcols['to_station_name'] == 'Wellesley St E / Yonge St Green P']

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
25327,742622,2017-01-21 18:38:00,2017-01-21 18:52:00,792.0,7064,51 Parliament St,7051,Wellesley St E / Yonge St Green P,Member,5,3,1,1,840.0,48.0,False,False
373798,1148169,2017-06-13 02:05:00,2017-06-13 04:16:00,7848.0,7051,Wellesley St E / Yonge St Green P,7051,Wellesley St E / Yonge St Green P,Casual,1,24,6,2,7860.0,12.0,False,False
464441,1251999,2017-06-30 21:27:00,2017-06-30 22:02:00,2114.0,7024,Dundonald St / Church St,7051,Wellesley St E / Yonge St Green P,Casual,4,26,6,2,2100.0,-14.0,False,False


In [126]:
get_relation(df_goodcols, 'to_station_id', 'to_station_name')


maxes: 1 2


'one-to-many'

In [150]:
df_goodcols = df_rand[df_rand['nan_row_flag'] == False]
df_goodcols_big = df[df['nan_row_flag'] == False]
df_goodcols_big.loc[:,'from_station_id'] = df_goodcols_big.loc[:,'from_station_id'].astype(int)
df_goodcols_big.loc[:,'to_station_id'] = df_goodcols_big.loc[:,'to_station_id'].astype(int)
df_goodcols['to_station_id'] = df_goodcols['to_station_id'].astype(int)
df_goodcols['from_station_id'] = df_goodcols['from_station_id'].astype(int)

/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [151]:
isOneToOne(df_goodcols, 'to_station_id', 'to_station_name')

True

In [152]:
isOneToOne(df_goodcols, 'from_station_id', 'from_station_name')

True

In [153]:
get_relation(df_goodcols, 'to_station_id', 'to_station_name')

maxes: 1 1


'one-to-one'

In [154]:
get_relation(df_goodcols, 'from_station_id', 'from_station_name')

maxes: 1 1


'one-to-one'

In [155]:
int(np.nan)

ValueError: cannot convert float NaN to integer

In [156]:
df_to_stn = df_goodcols.groupby('from_station_id')['from_station_name'].count().reset_index(name="count")

In [157]:
df_to_stn.head()

,from_station_id,count
0,7000,1
1,7002,1
2,7003,1
3,7006,2
4,7007,2


In [158]:
df_goodcols[df_goodcols['from_station_id'] == 7006]

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
318663,1084825,2017-06-01 17:29:00,2017-06-01 17:38:00,585.0,7006,Bay St / College St (East Side),7057,Simcoe St / Wellington St W,Member,3,22,6,2,540.0,-45.0,False,False
412438,1192488,2017-06-20 18:58:00,2017-06-20 19:06:00,471.0,7006,Bay St / College St (East Side),7042,Sherbourne St / Wellesley St,Member,1,25,6,2,480.0,9.0,False,False


In [159]:
df_good_format = df_goodcols.groupby(['from_station_id','from_station_name']).size().reset_index().rename(columns={0:'count'})

In [160]:
df_good_format.head()

,from_station_id,from_station_name,count
0,7000,Fort York Blvd / Capreol Crt,1
1,7002,St George St / Bloor St W,1
2,7003,Madison Ave / Bloor St W,1
3,7006,Bay St / College St (East Side),2
4,7007,College St W / Huron St,2


In [161]:
df_good_format_big = df_goodcols_big.groupby(['from_station_id','from_station_name']).size().reset_index().rename(columns={0:'count'})

In [162]:
df_good_format_big.head()

,from_station_id,from_station_name,count
0,7000,Fort York Blvd / Capreol Crt,3964
1,7001,Lower Jarvis St / The Esplanade,2458
2,7002,St George St / Bloor St W,3887
3,7003,Madison Ave / Bloor St W,3383
4,7004,University Ave / Elm St,2812


In [163]:
df_good_format_big[df_good_format_big['from_station_id'].duplicated(keep=False)].shape[0]

0

In [164]:
df_good_format_big[df_good_format_big['from_station_name'].duplicated(keep=False)]

,from_station_id,from_station_name,count


In [165]:
#df_good_format_big[df_good_format_big['to_station_id'].duplicated(keep=False)]

In [166]:
#df_good_format_big[df_good_format_big['to_station_name'].duplicated(keep=False)]

In [167]:
ids = set([])
badids = set([])
for x in df_good_format_big['from_station_id']:
    if x not in ids:
        ids.add(x)
        #print('added:', x)
    else:
        #print('error:', x)
        badids.add(x)
        

In [168]:
names = set([])
badnames = set([])
for x in df_good_format_big['from_station_name']:
    if x not in names:
        names.add(x)
        #print('added:', x)
    else:
        #print('error:', x)
        badnames.add(x)
    names = names.difference(badnames)

In [169]:
print(names)

{'80 Clinton St (North of College)', 'Bathurst St / Lennox St', 'East Liberty St / Pirandello St', 'Dundonald St / Church St', 'Bathurst St / Queens Quay W', 'Landsdowne Subway Green P', 'McGill St / Church St', 'Sterling Rd / Dundas St W', 'Danforth Ave / Lamb Ave', 'Wellington St W / Portland St', 'Danforth Ave / Barrington Ave', 'Riverdale Park South (Broadview Ave)', 'Danforth Ave / Westlake Ave', 'Danforth Ave / Gough Ave', 'Palmerston Ave / Vermont Ave', 'Trinity St /Front St E', 'Davenport Rd / Christie St', '161 Bleecker St (South of Wellesley)', 'Queen St E / Sackville St', 'Niagara St / Richmond St W', 'King St W / Tecumseth St', 'King St W / Bay St (East Side)', 'Elizabeth St / Edward St (Bus Terminal)', 'Dundas St E / Broadview Ave', 'Gerrard St E / River St', 'Dundas St E / Regent Park Blvd', 'Wellington St W / Stafford St', 'King St W / Bay St (West Side)', 'Queens Quay / Yonge St', 'Chester Station', 'Davenport Rd / Avenue Rd', 'Metro Hall Plaza', 'Dundas St / Yonge St',

In [170]:
print(badnames)

set()


In [171]:
#pd.Series(df.Letter.values,index=df.Position).to_dict()
from_dict = pd.Series(df_good_format_big['from_station_id'].values, index = df_good_format_big['from_station_name']).to_dict()

In [172]:
print(from_dict)

{'Fort York  Blvd / Capreol Crt': 7000, 'Lower Jarvis St / The Esplanade': 7001, 'St George St / Bloor St W': 7002, 'Madison Ave / Bloor St W': 7003, 'University Ave / Elm St': 7004, 'University Ave / King St W': 7005, 'Bay St / College St (East Side)': 7006, 'College St W / Huron St': 7007, "Wellesley St / Queen's Park Cres": 7008, 'King St E / Jarvis St': 7009, 'King St W / Spadina Ave': 7010, 'Wellington St W / Portland St': 7011, 'Elizabeth St / Edward St (Bus Terminal)': 7012, 'Scott St / The Esplanade': 7013, 'Sherbourne St / Carlton St (Allan Gardens)': 7014, 'King St W / Bay St (West Side)': 7015, 'Bay St / Queens Quay W (Ferry Terminal)': 7016, 'Widmer St / Adelaide St ': 7017, 'Bremner Blvd / Rees St': 7018, 'Temperance St / Yonge St': 7019, 'Phoebe St / Spadina Ave': 7020, 'Bay St / Albert St': 7021, 'Simcoe St / Queen St W': 7022, 'College St W / Borden St': 7023, 'Dundonald St / Church St': 7024, 'Ted Rogers Way / Bloor St E': 7025, 'Bay St / St. Joseph St': 7026, 'Beverly

In [173]:
from_dict_r = pd.Series(df_good_format_big['from_station_name'].values, index = df_good_format_big['from_station_id']).to_dict()

In [174]:
print(from_dict_r)

{7000: 'Fort York  Blvd / Capreol Crt', 7001: 'Lower Jarvis St / The Esplanade', 7002: 'St George St / Bloor St W', 7003: 'Madison Ave / Bloor St W', 7004: 'University Ave / Elm St', 7005: 'University Ave / King St W', 7006: 'Bay St / College St (East Side)', 7007: 'College St W / Huron St', 7008: "Wellesley St / Queen's Park Cres", 7009: 'King St E / Jarvis St', 7010: 'King St W / Spadina Ave', 7011: 'Wellington St W / Portland St', 7012: 'Elizabeth St / Edward St (Bus Terminal)', 7013: 'Scott St / The Esplanade', 7014: 'Sherbourne St / Carlton St (Allan Gardens)', 7015: 'King St W / Bay St (West Side)', 7016: 'Bay St / Queens Quay W (Ferry Terminal)', 7017: 'Widmer St / Adelaide St ', 7018: 'Bremner Blvd / Rees St', 7019: 'Temperance St / Yonge St', 7020: 'Phoebe St / Spadina Ave', 7021: 'Bay St / Albert St', 7022: 'Simcoe St / Queen St W', 7023: 'College St W / Borden St', 7024: 'Dundonald St / Church St', 7025: 'Ted Rogers Way / Bloor St E', 7026: 'Bay St / St. Joseph St', 7027: 'B

## Again for to stations

In [175]:
df_to_stns = df_goodcols_big.groupby(['to_station_id','to_station_name']).size().reset_index().rename(columns={0:'count'})

In [176]:
to_dict = pd.Series(df_to_stns['to_station_id'].values, index = df_to_stns['to_station_name']).to_dict()
to_dict_r = pd.Series(df_to_stns['to_station_name'].values, index = df_to_stns['to_station_id']).to_dict()

In [177]:
name_num_dict = from_dict.update(to_dict)
num_name_dict = from_dict_r.update(to_dict_r)

from_dict['Fort York  Blvd / Capreol Crt']

7000

In [178]:
bad_keys = []
for key in from_dict:
    if key in to_dict:
        if to_dict[key] != from_dict[key]:
            bad_keys.append(key)
            print(key)

In [179]:
bad_keys

[]

In [180]:
bad_keys2 = []
for key in from_dict_r:
    if key in to_dict_r:
        if to_dict_r[key] != from_dict_r[key]:
            bad_keys2_r.append(key)

In [181]:
df_good_format_big.shape

(200, 3)

In [182]:
len(to_dict_r)

200

In [183]:
len(from_dict_r)

200

In [184]:
len(from_dict)

200

In [185]:
df_goodcols_big['from_station_name'].nunique()

200

In [186]:
df_to_stns['to_station_name'].nunique()

200

In [187]:
df_to_stns['to_station_id'].nunique()

200

# Conclusion is the values are good
### test dictionary

In [188]:
    #df_badcols
    df_badcols.loc[:,'from_station_id'] = df_badcols.loc[:,'from_station_name'].map(from_dict)

In [189]:
df_badcols

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,day,week,month,quarter,check_time,time_delta,time_flag,nan_row_flag
469192,1257967,2017-07-01 19:39:00,2017-07-01 19:53:00,820.0,7033.0,Union Station,NaN,Trinity St /Front St E,Casual,5,26,7,3,840.0,20.0,False,True
473395,1262355,2017-07-02 15:35:00,2017-07-02 16:40:00,3898.0,7075.0,Queens Quay W / Dan Leckie Way,NaN,Jimmie Simpson Park (Queen St E),Casual,6,26,7,3,3900.0,2.0,False,True
474320,1263372,2017-07-02 17:25:00,2017-07-02 18:07:00,2550.0,7076.0,York St / Queens Quay W,NaN,Cherry St / Distillery Ln,Casual,6,26,7,3,2520.0,-30.0,False,True
476956,1266202,2017-07-03 09:48:00,2017-07-03 09:51:00,164.0,7011.0,Wellington St W / Portland St,NaN,King St W / Spadina Ave,Member,0,27,7,3,180.0,16.0,False,True
480300,1269749,2017-07-03 16:31:00,2017-07-03 16:46:00,943.0,7182.0,Sterling Rd / Dundas St W,NaN,King St W / Joe Shuster Way,Casual,0,27,7,3,900.0,-43.0,False,True
488369,1278604,2017-07-04 18:57:00,2017-07-04 19:10:00,742.0,7046.0,Niagara St / Richmond St W,NaN,Bay St / Albert St,Member,1,27,7,3,780.0,38.0,False,True
491712,1282306,2017-07-05 10:47:00,2017-07-05 10:52:00,335.0,7006.0,Bay St / College St (East Side),NaN,Beverly St / College St,Member,2,27,7,3,300.0,-35.0,False,True
493476,1284287,2017-07-05 15:04:00,2017-07-05 15:20:00,982.0,7032.0,Augusta Ave / Dundas St W,NaN,Yonge St / Aylmer Ave,Member,2,27,7,3,960.0,-22.0,False,True
495854,1286872,2017-07-05 17:54:00,2017-07-05 18:02:00,483.0,7011.0,Wellington St W / Portland St,NaN,Union Station,Member,2,27,7,3,480.0,-3.0,False,True
499405,1290649,2017-07-06 00:36:00,2017-07-06 00:55:00,1143.0,7126.0,Yonge St / Yorkville Ave,NaN,Bathurst St / Dundas St W,Member,3,27,7,3,1140.0,-3.0,False,True


In [190]:
from_dict['College St / Major St']

KeyError: 'College St / Major St'

In [191]:
from_dict

{'Fort York  Blvd / Capreol Crt': 7000,
 'Lower Jarvis St / The Esplanade': 7001,
 'St George St / Bloor St W': 7002,
 'Madison Ave / Bloor St W': 7003,
 'University Ave / Elm St': 7004,
 'University Ave / King St W': 7005,
 'Bay St / College St (East Side)': 7006,
 'College St W / Huron St': 7007,
 "Wellesley St / Queen's Park Cres": 7008,
 'King St E / Jarvis St': 7009,
 'King St W / Spadina Ave': 7010,
 'Wellington St W / Portland St': 7011,
 'Elizabeth St / Edward St (Bus Terminal)': 7012,
 'Scott St / The Esplanade': 7013,
 'Sherbourne St / Carlton St (Allan Gardens)': 7014,
 'King St W / Bay St (West Side)': 7015,
 'Bay St / Queens Quay W (Ferry Terminal)': 7016,
 'Widmer St / Adelaide St ': 7017,
 'Bremner Blvd / Rees St': 7018,
 'Temperance St / Yonge St': 7019,
 'Phoebe St / Spadina Ave': 7020,
 'Bay St / Albert St': 7021,
 'Simcoe St / Queen St W': 7022,
 'College St W / Borden St': 7023,
 'Dundonald St / Church St': 7024,
 'Ted Rogers Way / Bloor St E': 7025,
 'Bay St / St. 